For solving path dependences

In [14]:
import sys
import os

current_dir = %pwd

# Detect the parent directory dynamically
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

In [21]:
from src.main.utils import *

X, y = shuffle_data(np.array([[1, 2], [3, 4], [5, 6], [7, 8]]), np.array([10, 20, 30, 40]))



print(X)
print(y)

[[5 6]
 [3 4]
 [1 2]
 [7 8]]
[30 20 10 40]
